In [3]:

# different
"""one = {
    "blood": 20,
    "blood_sat": 0.8,
    "a": 24.2,
    "b": 1.6,
    "lipid_frac": 0.5,
    "water_frac": 0.5
}
one_height = 1

two = one.copy()
two['blood'] = 50
two['a'] = 30
two['b'] = 1.0
two_height = 1e8"""

# same

one = {
    "blood": 20,
    "blood_sat": 0.8,
    "a": 24.2,
    "b": 1.6,
    "lipid_frac": 0.5,
    "water_frac": 0.5
}
two = one.copy()


one_height = 1
two_height = 1e8

In [4]:
import pandas as pd
import random
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
from helpers.interpolate_extinction_coeffs import extinction_coeffs
from helpers.mci_files import create_layer_dict, createMciString

n = 1.4
g = 0.9
csf_scatter = 1.0
csf_abs = 0.0
wls = np.arange(740, 902, 2) 
_, (water_ex, oxy_ex, deoxy_ex, lipid_ex) = extinction_coeffs(wls)

add_postfix = lambda arr, postfix: [e + postfix for e in arr]
extract_keys = lambda sample, keys: [float(sample[e]) for e in keys]

csf_heights = [0.1, 0.2, 0.5, 1.0]

basename = 'same'

absorption = [e['water_frac']*55.5*water_ex + e['blood']*1e-6*(e['blood_sat']*oxy_ex + (1-e['blood_sat'])*deoxy_ex)+ e['lipid_frac']*lipid_ex for e in [one, two]]
scattering = [e['a']*(wls/500.0)**(-e['b'])/(1-g) for e in [one, two]]

layer_collections = []
for d in csf_heights:
    layers_arr = []
    for i in range(len(absorption[0])):
        layers_arr.append([
            create_layer_dict(n, absorption[0][i], scattering[0][i], g, one_height),
            create_layer_dict(n, csf_abs, csf_scatter, g, d),
            create_layer_dict(n, absorption[1][i], scattering[1][i], g, two_height)
        ])
    layer_collections.append(layers_arr)

file_names = [[f'{basename}_output/{basename}_{int(h*10)}_{i}' for i in range(len(absorption[0]))] for h in csf_heights]

for run, names, h in zip(layer_collections, file_names, csf_heights):
    with open(f"./sims/{basename}_variable_layers/{basename}_{int(h*10)}.mci", 'w') as f:
        f.write(createMciString(run, names, number_photons=1e8))